# Hyperparameter Tuning using HyperDrive

Importing all needed dependencies to complete the project.

In [1]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive_exp'

experiment=Experiment(ws, experiment_name)

In [4]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [14]:
# Create CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_V2',
                                                           vm_priority = 'dedicated', 
                                                           max_nodes=30)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [15]:
# create environment
environment = Environment(name="azure-env")
conda_dep = CondaDependencies()

# Needed packages and scripts
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_dep

## Hyperdrive Configuration

I have chosen a LogisticRegression classifier model to train the HyperDrive experiment. Since our target is to predict classification problem result either 0 or 1. The model uses the probability of a certain class to occur. Logistic regression uses a logistic model function which is a form of binary regression. The model is trained using 'train.py' script.

The used HyperDrive parameters:

- Parameter sampler 'RandomParameterSampling' holds the tuning hyperparameters (--C: Inverse of regularization, --max_iter: Maximum number of iterations) was passed to the HyperDriveConfig script.
  - Discrete values with 'choice' have been used for both tuned parameters '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000), '--max_iter': choice(10,50). RandomParameterSampling has been selected due to its fast performance, simple approach, and would provide random unbiased search in the overall population. In addition, it gives satisfactory results and supports the early termination policy of low-performance runs which results in saving resources. Grid Sampling can be used for exhaustive search over the search space if the budget was not an issue.
- Early termination policy has been added to the script then experiment submission.
  - BanditPolicy has been used with the parameters evaluation_interval=2 and slack_factor=0.1 as an early stopping policy to improve the performance of the computational resources by automatically terminating poorly and delayed performing runs. Bandit Policy ends runs if the primary metric is not within the specified slack factor/amount when compared with the highest performing run.


In [20]:
# Create an early termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

# Create different params that will be used during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000),
        '--max_iter': choice(10,50)
    }
)

# Create estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            arguments=['--data', dataset.id],
                            compute_target=amlcompute_cluster_name,
                            environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=30)

In [21]:
# Submit experiment
hd_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_991759b2-6e99-4e9d-9299-473eb49f633a
Web View: https://ml.azure.com/runs/HD_991759b2-6e99-4e9d-9299-473eb49f633a?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-248655/workspaces/quick-starts-ws-248655&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-01-08T09:43:35.009773][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space
[2024-01-08T09:43:36.1060043Z][SCHEDULER][INFO]Scheduling job, id='HD_991759b2-6e99-4e9d-9299-473eb49f633a_0' 
[2024-01-08T09:43:36.2041952Z][SCHEDULER][INFO]Scheduling job, id='HD_991759b2-6e99-4e9d-9299-473eb49f633a_1' 
[2024-01-08T09:43:36.3575797Z][SCHEDULER][INFO]Scheduling job, id='HD_991759b2-6e99-4e9d-9299-473eb49f633a_2' 
[2024-01-08T09:43:36.3985231Z][SCHEDULER][INFO]Scheduling job, id='HD_991759b2-6e99-4e9d-9299-473eb49f633a_3' 
[2024-01-08T09:43:36.5568158Z][SCHEDULER][INFO]Scheduling job, id='HD_991759b2-6e99-4e9d-9299-473eb49f633a_4' 
[2024-01-0

{'runId': 'HD_991759b2-6e99-4e9d-9299-473eb49f633a',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-01-08T09:43:34.078348Z',
 'endTimeUtc': '2024-01-08T09:49:07.791213Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5e2df888-bed5-4582-b032-80a7335c490c',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '16',
  'best_child_run_id': 'HD_991759b2-6e99-4e9d-9299-473eb49f633a_1',
  'score': '0.7708333333333334',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_991759b2-6e99-4e9d-9299-473eb49f633a_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

In [24]:
hd_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive_exp,HD_991759b2-6e99-4e9d-9299-473eb49f633a,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [29]:
# Retrieve and save best model
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_metrics()

{'Regularization Strength:': 1.0,
 'Max iterations:': 50,
 'Accuracy': 0.7708333333333334}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Registring the best model
model = best_run.register_model(model_name='hd-best-model', model_path='outputs/model.joblib')


In [ ]:
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=environment)
service_name = 'hd-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = [{"Pregnancies": 6, 
 "Glucose": 148, 
 "BloodPressure": 72, 
 "SkinThickness": 35, 
 "Insulin": 0, 
 "BMI": 33.5, 
 "DiabetesPedigreeFunction": 0.627, 
 "Age": 50},
        
{"Pregnancies": 1, 
 "Glucose": 85, 
 "BloodPressure": 66, 
 "SkinThickness": 29, 
 "Insulin": 20, 
 "BMI": 26.5, 
 "DiabetesPedigreeFunction": 0.351, 
 "Age": 31}]

print(data)


In [ ]:
import json


input_payload = json.dumps({
    'data': df[0:2].values.tolist(),
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()